In [1]:
import os
import json
import pandas as pd
import collections

In [2]:
import os, json
isbns = set()
for dirpath, _, filenames in os.walk('./datasets/Training'):
    for fn in filenames:
        if fn.lower().endswith('.json'):
            with open(os.path.join(dirpath, fn), encoding='utf-8-sig') as f:
                d = json.load(f)
            books = d if isinstance(d, list) else [d]
            for book in books:
                isbns.add(str(book.get('isbn','')).strip())
print(f"원본 폴더 기준 고유권수 : {len(isbns)}")  # 원본 폴더 기준 고유권수

원본 폴더 기준 고유권수 : 1446


In [3]:
import os, json
isbns = set()
for dirpath, _, filenames in os.walk('./datasets/Validation'):
    for fn in filenames:
        if fn.lower().endswith('.json'):
            with open(os.path.join(dirpath, fn), encoding='utf-8-sig') as f:
                d = json.load(f)
            books = d if isinstance(d, list) else [d]
            for book in books:
                isbns.add(str(book.get('isbn','')).strip())
print(f"원본 폴더 기준 고유권수 : {len(isbns)}")  # 원본 폴더 기준 고유권수

원본 폴더 기준 고유권수 : 286


In [4]:
root_dir = './datasets'

In [5]:
json_files = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.lower().endswith('.json'):
            json_files.append(os.path.join(dirpath, filename))

In [6]:
isbn_title_set = set()
isbn_meta_dict = {}

for filepath in json_files:
    try:
        with open(filepath, 'r', encoding='utf-8-sig') as f:
            data = json.load(f)
        if isinstance(data, list):  # 혹시 리스트형이면
            items = data
        else:
            items = [data]
        for book in items:
            isbn = str(book.get('isbn', '')).strip()
            title = book.get('title', '').strip()
            if isbn and title:
                isbn_title_set.add((isbn, title))
                # 메타만 남기기
                meta = {
                    "isbn": isbn,
                    "title": title,
                    "author": book.get('author', '').strip(),
                    "illustrator": book.get('illustrator', '').strip(),
                    "readAge": book.get('readAge', '').strip(),
                    "publishedYear": book.get('publishedYear', ''),
                    "publisher": book.get('publisher', '').strip(),
                    "classification": book.get('classification', '').strip()
                }
                isbn_meta_dict[isbn] = meta
    except Exception as e:
        print(f"Error: {filepath}, {e}")

### csv

In [7]:
# 1. isbn + title csv
df_titles = pd.DataFrame(list(isbn_title_set), columns=['isbn', 'title']).sort_values('isbn')
df_titles.to_csv(os.path.join(root_dir, 'isbn_title_only.csv'), index=False, encoding='utf-8-sig')
print(f"1. isbn-title csv 저장 ({len(df_titles)} 행)")

1. isbn-title csv 저장 (1732 행)


### json

In [8]:
# 2. isbn별 메타데이터 json 파일
meta_dir = os.path.join(root_dir, 'isbn_meta')
os.makedirs(meta_dir, exist_ok=True)
for isbn, meta in isbn_meta_dict.items():
    with open(os.path.join(meta_dir, f"{isbn}.json"), 'w', encoding='utf-8-sig') as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)
print(f"2. isbn별 메타 json 저장 ({len(isbn_meta_dict)} 파일)")

2. isbn별 메타 json 저장 (1732 파일)


### meta

In [9]:
# 3. 전체 통합 메타 json
all_meta_path = os.path.join(root_dir, 'all_isbn_meta.json')
with open(all_meta_path, 'w', encoding='utf-8-sig') as f:
    json.dump(list(isbn_meta_dict.values()), f, ensure_ascii=False, indent=2)
print(f"3. 전체 통합 메타 json 저장 ({len(isbn_meta_dict)}권)")

3. 전체 통합 메타 json 저장 (1732권)


# 데이터 전처리

### Training

In [10]:
import os
import json
import shutil
import pandas as pd
from collections import defaultdict

# --- 경로 설정 ---
root_dir = './datasets/Training'
sublabel_dir = './datasets/Sublabel'
converted_root = './converted/training'
converted_sublabel = './converted/Sublabel'
json_dir = os.path.join(converted_root, 'json')
os.makedirs(json_dir, exist_ok=True)
os.makedirs(converted_sublabel, exist_ok=True)

# --- 1. isbn별 메타/질문 집계 ---
isbn_info = {}
isbn_explicit = defaultdict(bool)

json_files = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.lower().endswith('.json'):
            json_files.append(os.path.join(dirpath, filename))

for src_path in json_files:
    with open(src_path, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
    books = data if isinstance(data, list) else [data]
    for book in books:
        isbn = str(book.get('isbn', '')).strip()
        title = book.get('title', '').strip()
        if not isbn:
            continue
        # 메타 정보 최초 1회만 기록
        if isbn not in isbn_info:
            meta = {"isbn": isbn, "title": title}
            for key in ["author", "illustrator", "readAge", "publishedYear", "publisher", "classification"]:
                if key in book and book[key]:
                    meta[key] = str(book[key]).strip()
            isbn_info[isbn] = meta
        # 명시적 질문 있는지 체크
        for para in book.get('paragraphInfo', []):
            for qa in para.get('queAnsPairInfo', []):
                if '명시적' in qa.get('ansType', ''):
                    isbn_explicit[isbn] = True

isbn_all_set = set(isbn_info.keys())
isbn_explicit_set = set(isbn for isbn, flag in isbn_explicit.items() if flag)
isbn_removed_set = isbn_all_set - isbn_explicit_set

print(f"전체 isbn 고유권수: {len(isbn_all_set)}")
print(f"명시적 질문 1개 이상 있는 책 권수: {len(isbn_explicit_set)}")
print(f"명시적 질문 0개인 책 권수: {len(isbn_removed_set)}")

# --- 2. 변환 json/meta ---
meta_list = []
meta_dict = {}

for src_path in json_files:
    with open(src_path, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
    books = data if isinstance(data, list) else [data]
    new_books = []
    for book in books:
        isbn = str(book.get('isbn', '')).strip()
        if isbn not in isbn_explicit_set:
            continue
        # 명시적 질문만 남긴다
        book_copy = dict(book)
        new_paragraphs = []
        for para in book.get('paragraphInfo', []):
            new_para = dict(para)
            new_qapairs = []
            for qa in para.get('queAnsPairInfo', []):
                if '명시적' in qa.get('ansType', ''):
                    new_qapairs.append(qa)
            new_para['queAnsPairInfo'] = new_qapairs
            new_para['queAnsPairInfoCount'] = len(new_qapairs)
            new_paragraphs.append(new_para)
        book_copy['paragraphInfo'] = new_paragraphs
        new_books.append(book_copy)
    if new_books:
        rel_path = os.path.relpath(src_path, root_dir)
        save_path = os.path.join(converted_root, rel_path)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        with open(save_path, 'w', encoding='utf-8-sig') as f:
            if isinstance(data, list):
                json.dump(new_books, f, ensure_ascii=False, indent=2)
            else:
                json.dump(new_books[0], f, ensure_ascii=False, indent=2)

for isbn in sorted(isbn_explicit_set):
    meta_dict[isbn] = isbn_info[isbn]
    meta_list.append(isbn_info[isbn])

# --- 3. Sublabel 복사(하위폴더 전체에서 isbn에 맞는 파일 찾기) ---
def find_sublabel_file(sublabel_dir, isbn):
    for dirpath, _, filenames in os.walk(sublabel_dir):
        for filename in filenames:
            if filename.endswith(f"{isbn}.json"):
                return os.path.join(dirpath, filename)
    return None

copy_ok, copy_fail = 0, 0
for isbn in isbn_explicit_set:
    src_full = find_sublabel_file(sublabel_dir, isbn)
    dst_full = os.path.join(converted_sublabel, f"{isbn}.json")
    if src_full and os.path.exists(src_full):
        shutil.copy2(src_full, dst_full)
        copy_ok += 1
    else:
        print(f"[Sublabel 없음] {isbn}")
        copy_fail += 1
print(f"Sublabel 복사: {copy_ok}개 성공, {copy_fail}개 실패")

# --- 4. 결과 저장 ---
meta_simple_path = os.path.join(json_dir, 'book_titles_by_isbn.json')
with open(meta_simple_path, 'w', encoding='utf-8-sig') as f:
    json.dump([{"isbn": m["isbn"], "title": m["title"]} for m in meta_list], f, ensure_ascii=False, indent=2)

meta_all_path = os.path.join(json_dir, 'book_meta_all.json')
with open(meta_all_path, 'w', encoding='utf-8-sig') as f:
    json.dump(meta_list, f, ensure_ascii=False, indent=2)

meta_books_dir = os.path.join(json_dir, 'books_by_isbn')
os.makedirs(meta_books_dir, exist_ok=True)
for isbn, meta in meta_dict.items():
    meta_path = os.path.join(meta_books_dir, f'{isbn}.json')
    with open(meta_path, 'w', encoding='utf-8-sig') as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

removed_path = os.path.join(json_dir, 'removed_books.csv')
pd.DataFrame([isbn_info[i] for i in sorted(isbn_removed_set)]).to_csv(removed_path, index=False, encoding='utf-8-sig')

print(f"[최종] 전체 고유 isbn: {len(isbn_all_set)}")
print(f"[최종] 명시적 질문 있는 책: {len(isbn_explicit_set)}")
print(f"[최종] 명시적 질문 없는 책: {len(isbn_removed_set)}")
print(f"모든 전처리/저장 완료!")

전체 isbn 고유권수: 1446
명시적 질문 1개 이상 있는 책 권수: 1443
명시적 질문 0개인 책 권수: 3
[Sublabel 없음] 9791159420214
[Sublabel 없음] 9791186922972
[Sublabel 없음] 9788961914314
Sublabel 복사: 1440개 성공, 3개 실패
[최종] 전체 고유 isbn: 1446
[최종] 명시적 질문 있는 책: 1443
[최종] 명시적 질문 없는 책: 3
모든 전처리/저장 완료!


### Validation

In [11]:
import os
import json
import shutil
import pandas as pd
from collections import defaultdict

# --- 경로 설정 ---
root_dir = './datasets/Validation'
sublabel_dir = './datasets/Sublabel'
converted_root = './converted/validation'
converted_sublabel = './converted/Sublabel'
json_dir = os.path.join(converted_root, 'json')
os.makedirs(json_dir, exist_ok=True)
os.makedirs(converted_sublabel, exist_ok=True)

# --- 1. isbn별 메타/질문 집계 ---
isbn_info = {}
isbn_explicit = defaultdict(bool)

json_files = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.lower().endswith('.json'):
            json_files.append(os.path.join(dirpath, filename))

for src_path in json_files:
    with open(src_path, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
    books = data if isinstance(data, list) else [data]
    for book in books:
        isbn = str(book.get('isbn', '')).strip()
        title = book.get('title', '').strip()
        if not isbn:
            continue
        # 메타 정보 최초 1회만 기록
        if isbn not in isbn_info:
            meta = {"isbn": isbn, "title": title}
            for key in ["author", "illustrator", "readAge", "publishedYear", "publisher", "classification"]:
                if key in book and book[key]:
                    meta[key] = str(book[key]).strip()
            isbn_info[isbn] = meta
        # 명시적 질문 있는지 체크
        for para in book.get('paragraphInfo', []):
            for qa in para.get('queAnsPairInfo', []):
                if '명시적' in qa.get('ansType', ''):
                    isbn_explicit[isbn] = True

isbn_all_set = set(isbn_info.keys())
isbn_explicit_set = set(isbn for isbn, flag in isbn_explicit.items() if flag)
isbn_removed_set = isbn_all_set - isbn_explicit_set

print(f"전체 isbn 고유권수: {len(isbn_all_set)}")
print(f"명시적 질문 1개 이상 있는 책 권수: {len(isbn_explicit_set)}")
print(f"명시적 질문 0개인 책 권수: {len(isbn_removed_set)}")

# --- 2. 변환 json/meta ---
meta_list = []
meta_dict = {}

for src_path in json_files:
    with open(src_path, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
    books = data if isinstance(data, list) else [data]
    new_books = []
    for book in books:
        isbn = str(book.get('isbn', '')).strip()
        if isbn not in isbn_explicit_set:
            continue
        # 명시적 질문만 남긴다
        book_copy = dict(book)
        new_paragraphs = []
        for para in book.get('paragraphInfo', []):
            new_para = dict(para)
            new_qapairs = []
            for qa in para.get('queAnsPairInfo', []):
                if '명시적' in qa.get('ansType', ''):
                    new_qapairs.append(qa)
            new_para['queAnsPairInfo'] = new_qapairs
            new_para['queAnsPairInfoCount'] = len(new_qapairs)
            new_paragraphs.append(new_para)
        book_copy['paragraphInfo'] = new_paragraphs
        new_books.append(book_copy)
    if new_books:
        rel_path = os.path.relpath(src_path, root_dir)
        save_path = os.path.join(converted_root, rel_path)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        with open(save_path, 'w', encoding='utf-8-sig') as f:
            if isinstance(data, list):
                json.dump(new_books, f, ensure_ascii=False, indent=2)
            else:
                json.dump(new_books[0], f, ensure_ascii=False, indent=2)

for isbn in sorted(isbn_explicit_set):
    meta_dict[isbn] = isbn_info[isbn]
    meta_list.append(isbn_info[isbn])

# --- 3. Sublabel 복사(하위폴더 전체에서 isbn에 맞는 파일 찾기) ---
def find_sublabel_file(sublabel_dir, isbn):
    for dirpath, _, filenames in os.walk(sublabel_dir):
        for filename in filenames:
            if filename.endswith(f"{isbn}.json"):
                return os.path.join(dirpath, filename)
    return None

copy_ok, copy_fail = 0, 0
for isbn in isbn_explicit_set:
    src_full = find_sublabel_file(sublabel_dir, isbn)
    dst_full = os.path.join(converted_sublabel, f"{isbn}.json")
    if src_full and os.path.exists(src_full):
        shutil.copy2(src_full, dst_full)
        copy_ok += 1
    else:
        print(f"[Sublabel 없음] {isbn}")
        copy_fail += 1
print(f"Sublabel 복사: {copy_ok}개 성공, {copy_fail}개 실패")

# --- 4. 결과 저장 ---
meta_simple_path = os.path.join(json_dir, 'book_titles_by_isbn.json')
with open(meta_simple_path, 'w', encoding='utf-8-sig') as f:
    json.dump([{"isbn": m["isbn"], "title": m["title"]} for m in meta_list], f, ensure_ascii=False, indent=2)

meta_all_path = os.path.join(json_dir, 'book_meta_all.json')
with open(meta_all_path, 'w', encoding='utf-8-sig') as f:
    json.dump(meta_list, f, ensure_ascii=False, indent=2)

meta_books_dir = os.path.join(json_dir, 'books_by_isbn')
os.makedirs(meta_books_dir, exist_ok=True)
for isbn, meta in meta_dict.items():
    meta_path = os.path.join(meta_books_dir, f'{isbn}.json')
    with open(meta_path, 'w', encoding='utf-8-sig') as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

removed_path = os.path.join(json_dir, 'removed_books.csv')
pd.DataFrame([isbn_info[i] for i in sorted(isbn_removed_set)]).to_csv(removed_path, index=False, encoding='utf-8-sig')

print(f"[최종] 전체 고유 isbn: {len(isbn_all_set)}")
print(f"[최종] 명시적 질문 있는 책: {len(isbn_explicit_set)}")
print(f"[최종] 명시적 질문 없는 책: {len(isbn_removed_set)}")
print(f"모든 전처리/저장 완료!")


전체 isbn 고유권수: 286
명시적 질문 1개 이상 있는 책 권수: 284
명시적 질문 0개인 책 권수: 2
[Sublabel 없음] 9791159420160
[Sublabel 없음] 9791159420054
[Sublabel 없음] 9791128208904
Sublabel 복사: 281개 성공, 3개 실패
[최종] 전체 고유 isbn: 286
[최종] 명시적 질문 있는 책: 284
[최종] 명시적 질문 없는 책: 2
모든 전처리/저장 완료!


In [5]:
# 전체 JSON 파일 개수 세기
json_files = []

for root, dirs, files in os.walk("./converted/Sublabel"):
    for file in files:
        if file.endswith(".json"):
            json_files.append(os.path.join(root, file))

print(f"📂 전체 JSON 파일 수: {len(json_files)}")

📂 전체 JSON 파일 수: 1721


### Unpacking

import os
import shutil

base_dir = './converted/sublabel'

# 이동된 파일 수를 세기 위한 변수
moved_files_count = 0

# 01.원천데이터 폴더가 존재하는지 확인
if not os.path.isdir(base_dir):
    print(f"오류: 지정된 경로 '{base_dir}'가 존재하지 않거나 폴더가 아닙니다.")
else:
    print(f"'{base_dir}' 폴더에서 .json 파일 이동을 시작합니다.")

    for root, dirs, files in os.walk(base_dir, topdown=False):
        if root == base_dir:
            continue
        for file_name in files:
            if file_name.endswith('.json'):
                source_path = os.path.join(root, file_name)
                destination_path = os.path.join(base_dir, file_name)

                # 파일명 충돌 방지: 이미 대상 폴더에 동일한 파일명이 존재할 경우 처리
                if os.path.exists(destination_path):
                    # 충돌 해결 전략: (3) 이름 변경
                    base, ext = os.path.splitext(file_name)
                    counter = 1
                    new_file_name = f"{base}_{counter}{ext}"
                    while os.path.exists(os.path.join(base_dir, new_file_name)):
                        counter += 1
                        new_file_name = f"{base}_{counter}{ext}"
                    
                    destination_path = os.path.join(base_dir, new_file_name)
                    print(f"파일명 충돌: '{file_name}' -> '{new_file_name}'으로 변경하여 이동합니다.")
                
                try:
                    shutil.move(source_path, destination_path)
                    print(f"'{source_path}' -> '{destination_path}' (이동 완료)")
                    moved_files_count += 1
                except Exception as e:
                    print(f"'{source_path}' 이동 중 오류 발생: {e}")
        
        # 파일 이동 후, 현재 하위 디렉토리(root)가 비어있으면 삭제 
        try:
            if not os.listdir(root):
                os.rmdir(root)
                print(f"빈 디렉토리 삭제: '{root}'")
        except OSError as e:
            print(f"빈 디렉토리 '{root}' 삭제 중 오류 발생 (아마도 비어있지 않거나 권한 문제): {e}")

    print(f"\n모든 .json 파일 이동이 완료되었습니다. 총 {moved_files_count}개의 파일이 이동되었습니다.")

### Lookup dataset

In [28]:
import os

def check_sublabel_completeness(converted_base_dir):
    """
    training, validation 경로의 .json 파일명들을 sublabel 경로의 .json 파일명과 비교하여
    누락된 목록과 개수를 출력합니다.

    Args:
        converted_base_dir (str): 'converted' 폴더의 절대 또는 상대 경로.
                                  예: './converted' 또는 'C:/Users/user/project/converted'
    """
    training_dir = os.path.join(converted_base_dir, 'training')
    validation_dir = os.path.join(converted_base_dir, 'validation')
    sublabel_dir = os.path.join(converted_base_dir, 'sublabel')

    # 1. training 및 validation 경로의 모든 JSON 파일명 수집 (중복 제거)
    all_source_filenames = set()

    # training 디렉토리 탐색
    print(f"'{training_dir}' 에서 .json 파일 목록을 수집 중...")
    if not os.path.isdir(training_dir):
        print(f"경고: '{training_dir}' 경로가 존재하지 않습니다.")
    else:
        for root, _, files in os.walk(training_dir):
            for file_name in files:
                if file_name.endswith('.json'):
                    all_source_filenames.add(file_name)
        print(f"'{training_dir}' 에서 총 {len(all_source_filenames)}개의 고유 .json 파일명 수집 완료.")


    # validation 디렉토리 탐색 (기존 all_source_filenames에 추가)
    print(f"'{validation_dir}' 에서 .json 파일 목록을 수집 중...")
    if not os.path.isdir(validation_dir):
        print(f"경고: '{validation_dir}' 경로가 존재하지 않습니다.")
    else:
        current_source_count = len(all_source_filenames)
        for root, _, files in os.walk(validation_dir):
            for file_name in files:
                if file_name.endswith('.json'):
                    all_source_filenames.add(file_name)
        print(f"'{validation_dir}' 에서 추가로 {len(all_source_filenames) - current_source_count}개의 고유 .json 파일명 수집 완료.")

    total_source_count = len(all_source_filenames)
    print(f"\n총 (training + validation) 고유 .json 파일 개수: {total_source_count}개")

    # 2. sublabel 경로의 모든 JSON 파일명 수집
    all_sublabel_filenames = set()
    print(f"\n'{sublabel_dir}' 에서 .json 파일 목록을 수집 중...")
    if not os.path.isdir(sublabel_dir):
        print(f"오류: '{sublabel_dir}' 경로가 존재하지 않습니다. 이 폴더가 없으면 비교할 수 없습니다.")
        return # sublabel 폴더가 없으면 함수 종료

    for root, _, files in os.walk(sublabel_dir):
        for file_name in files:
            if file_name.endswith('.json'):
                all_sublabel_filenames.add(file_name)

    total_sublabel_count = len(all_sublabel_filenames)
    print(f"'{sublabel_dir}' 에서 총 {total_sublabel_count}개의 고유 .json 파일명 수집 완료.")

    # 3. 비교 및 결과 출력
    # training/validation에는 있지만 sublabel에는 없는 파일
    missing_in_sublabel = all_source_filenames - all_sublabel_filenames
    
    print("\n--- 비교 결과 ---")
    print(f"Training/Validation 총 고유 파일 개수: {total_source_count}개")
    print(f"Sublabel 총 고유 파일 개수: {total_sublabel_count}개")

    if not missing_in_sublabel:
        print("\n✅ 모든 Training/Validation 파일이 Sublabel에 존재합니다!")
    else:
        print(f"\n❌ Sublabel에 없는 Training/Validation 파일 개수: {len(missing_in_sublabel)}개")
        print("\nSublabel에 없는 파일 목록:")
        # 정렬하여 출력하면 보기 좋습니다.
        for filename in sorted(list(missing_in_sublabel)):
            print(f"- {filename}")

# --- 함수 호출 ---
# 여기에 'converted' 폴더의 실제 경로를 입력하세요.
# 예: converted_base_directory = 'C:/Users/사용자명/내프로젝트/converted'
# 또는 현재 주피터 노트북 파일이 'converted' 폴더와 같은 레벨에 있다면:
converted_base_directory = './converted' 

check_sublabel_completeness(converted_base_directory)

'./converted\training' 에서 .json 파일 목록을 수집 중...
'./converted\training' 에서 총 1443개의 고유 .json 파일명 수집 완료.
'./converted\validation' 에서 .json 파일 목록을 수집 중...
'./converted\validation' 에서 추가로 284개의 고유 .json 파일명 수집 완료.

총 (training + validation) 고유 .json 파일 개수: 1727개

'./converted\sublabel' 에서 .json 파일 목록을 수집 중...
'./converted\sublabel' 에서 총 2342개의 고유 .json 파일명 수집 완료.

--- 비교 결과 ---
Training/Validation 총 고유 파일 개수: 1727개
Sublabel 총 고유 파일 개수: 2342개

❌ Sublabel에 없는 Training/Validation 파일 개수: 4개

Sublabel에 없는 파일 목록:
- 03_02T_02S_9788961914314.json
- 03_03T_03S_9791128208904.json
- 03_03T_03S_9791159420054_.json
- 03_03T_03S_9791159420160_.json


### Check token size

In [ ]:
import json, sys, re
from pathlib import Path
from tqdm import tqdm

BASE = Path("./converted/formatted")
SPLITS = ["train", "val"]

def load_json(fp):
    with open(fp, "r", encoding="utf-8") as f:
        return json.load(f)

def fail(msg, errors, fp):
    errors.append(f"{fp} :: {msg}")

def validate_file(fp):
    errors=[]
    try:
        js = load_json(fp)
    except Exception as e:
        return [f"{fp} :: JSON load error – {e}"]

    if "data"   not in js: fail("'data' key missing", errors, fp)
    if "version" not in js: fail("'version' key missing", errors, fp)

    for d_i, data in enumerate(js.get("data", [])):
        if "title" not in data: fail(f"[data[{d_i}]] 'title' missing", errors, fp)
        for p_i, para in enumerate(data.get("paragraphs", [])):
            context = para.get("context")
            if context is None: fail(f"[{d_i}][{p_i}] context missing", errors, fp)
            for q_i, qa in enumerate(para.get("qas", [])):
                qid = qa.get("id", "<no-id>")
                if "question" not in qa:  fail(f"{qid} question missing", errors, fp)
                if "is_impossible" not in qa:
                    fail(f"{qid} is_impossible missing", errors, fp)
                    continue
                imps = qa["is_impossible"]

                ans_list = qa.get("answers", [])
                if imps and ans_list:
                    fail(f"{qid} marked impossible but answers provided", errors, fp)
                if not imps and not ans_list:
                    fail(f"{qid} possible but answers empty", errors, fp)

                # answer-context 정합성
                for a_i, ans in enumerate(ans_list):
                    text = ans.get("text")
                    pos  = ans.get("answer_start")
                    if text is None or pos is None:
                        fail(f"{qid} answer[{a_i}] missing field", errors, fp)
                        continue
                    if context is not None and context[pos:pos+len(text)] != text:
                        snippet = context[pos:pos+len(text)]
                        fail(f"{qid} answer mismatch (ctx:'{snippet}' vs ans:'{text}')", errors, fp)
    return errors

def validate_split(split):
    print(f"\n🔍 VALIDATE {split.upper()}")
    split_dir = BASE / split
    issues=[]
    for fp in tqdm(list(split_dir.glob("*.json"))):
        issues.extend(validate_file(fp))
    if issues:
        print(f"❌ {len(issues)} issue(s) found in {split} files")
        for msg in issues[:20]:   # 처음 20개만 미리보기
            print("   •", msg)
    else:
        print("✅ all {split} files are valid")

for s in SPLITS:
    validate_split(s)

print("\n🟢 검증 완료")

### formatting

In [8]:
# ╔═══════════════╗
# ║  🟢  Cell 1   ║  KorQuAD 형식 말뭉치 생성 스크립트
# ╚═══════════════╝
import json, re, hashlib
from pathlib import Path
from typing import Union, Dict, List
from tqdm.auto import tqdm

# ───────── 경로 설정 ─────────
ROOT      = Path('./converted')
TRAIN_LB  = ROOT/'training'  /'02.라벨링데이터'
VAL_LB    = ROOT/'validation'/'02.라벨링데이터'
SUB_DIR   = ROOT/'sublabel'
OUT_DIR   = ROOT/'formatted'; OUT_DIR.mkdir(exist_ok=True)

# ───────── ISBN 추출 ─────────
ISBN_RE = re.compile(r'(\d{9,13}X?)_?(?=\.json$)')
def isbn_from_name(src: Union[str, Path]) -> str:
    name = src.name if hasattr(src, 'name') else str(src)
    m = ISBN_RE.search(name)
    if not m:
        raise ValueError(f'ISBN not found: {name}')
    return m.group(1)

# ───────── 데이터 로딩 ─────────
def load_dir(p: Path) -> Dict[str, dict]:
    return {isbn_from_name(f): json.loads(f.read_text('utf-8-sig')) for f in p.glob('*.json')}

def load_sublabels(p: Path) -> Dict[str, str]:
    tmp: Dict[str, List[str]] = {}
    for f in p.glob('*.json'):
        isbn = isbn_from_name(f)
        txt = json.loads(f.read_text('utf-8-sig'))['text']
        tmp.setdefault(isbn, []).append(txt)
    return {k: '\n\n'.join(v) for k, v in tmp.items()}

print("📄  loading …")
train_lbl = load_dir(TRAIN_LB)
val_lbl   = load_dir(VAL_LB)
sub_txt   = load_sublabels(SUB_DIR)
print(f"  train label {len(train_lbl)} | val label {len(val_lbl)} | sub {len(sub_txt)}")

# ───────── KorQuAD entry 생성 ─────────
def entry(isbn: str, lbl: dict, context: str) -> tuple[dict, list]:
    qas, missed = [], []
    for qa in lbl["paragraphInfo"][0]["queAnsPairInfo"]:
        qid = f'{isbn}-{hashlib.md5(qa["question"].encode()).hexdigest()[:8]}'
        ans = qa["ansM1"]
        if not ans: continue
        start = context.find(ans)
        if start == -1:
            missed.append((qa["question"], ans))
            continue
        qas.append({
            "id": qid,
            "question": qa["question"],
            "answers": [{"text": ans, "answer_start": start}]
        })
    return {
        "title": lbl["title"],
        "paragraphs": [{
            "context": context,
            "qas": qas
        }]
    }, missed

# ───────── Split 생성 및 저장 ─────────
def make_split(lbl_pool: Dict[str, dict], split: str):
    data, skip_isbn = [], []
    total_qas, missed_qas_total = 0, 0
    missed_detail = []

    for isbn, lbl in tqdm(lbl_pool.items(), desc=f"{split.upper()}"):
        context = sub_txt.get(isbn)
        if not context:
            skip_isbn.append(isbn)
            continue

        record, missed = entry(isbn, lbl, context)
        total_qas += sum(1 for _ in lbl["paragraphInfo"][0]["queAnsPairInfo"])
        missed_qas_total += len(missed)
        if record["paragraphs"][0]["qas"]:
            data.append(record)
        if missed:
            missed_detail.append((isbn, missed))

    out_path = OUT_DIR / f"{split}.json"
    json.dump({"version": "v1.0", "data": data}, out_path.open('w', encoding='utf-8'), ensure_ascii=False, indent=2)

    print(f"\n✅ {out_path.name} — books: {len(data)}  skipped_isbn: {len(skip_isbn)}")
    print(f"🔹 총 QA 수: {total_qas} | 미포함 QA 수: {missed_qas_total}")
    if skip_isbn:
        print(f"   ↪️  말뭉치 누락 ISBN 예시: {skip_isbn[:5]}")
    if missed_detail:
        print("   ↪️  정답 미포함 QA 예시:")
        for isbn, qa_list in missed_detail[:3]:
            for q, a in qa_list[:2]:
                print(f"      • [{isbn}] Q: {q} / A: {a}")

# ───────── 실행 ─────────
make_split(train_lbl, 'train')
make_split(val_lbl,   'val')


📄  loading …
  train label 1443 | val label 284 | sub 1725


TRAIN:   0%|          | 0/1443 [00:00<?, ?it/s]


✅ train.json — books: 1429  skipped_isbn: 1
🔹 총 QA 수: 2521 | 미포함 QA 수: 85
   ↪️  말뭉치 누락 ISBN 예시: ['9788961914314']
   ↪️  정답 미포함 QA 예시:
      • [9791128212765] Q: 무서운 뱀인줄 알고 어떻게 하자고 했나요? / A: 도망가자.
      • [9791128216725] Q: 여우가 우주선을 타고 있는 걸 본 친구들은 뭐라고 했나요? / A: 우리도 태워 줘.
      • [9791128216985] Q: 도토리 달을 먹은 다람쥐는 어떻게 하나요? / A: 다람쥐가 웃어요. 


VAL:   0%|          | 0/284 [00:00<?, ?it/s]


✅ val.json — books: 276  skipped_isbn: 1
🔹 총 QA 수: 512 | 미포함 QA 수: 22
   ↪️  말뭉치 누락 ISBN 예시: ['9791128208904']
   ↪️  정답 미포함 QA 예시:
      • [9791165432423] Q: 친구와 함께 그네를 탔을 때 어떤 기분을 느낄까요? / A: 신난다.
      • [9791165434724] Q: 아이의 배변을 도와주려고 하는 이는 누구인가요? / A: 야옹이
      • [9791165435950] Q: 책은 혼자 보는 것보다 어떻게 보는 것이 더 재미있을까요? / A: 친구와 함께 보면 더 재미나요.


### validate

In [9]:
# ╔═══════════════╗
# ║  🟢  Cell 2   ║  상세 검증 스크립트
# ╚═══════════════╝

import json, textwrap, statistics
from pathlib import Path

def validate(split: str, root: Path = Path("./converted/formatted")):
    path = root / f"{split}.json"
    if not path.exists():
        print(f"{split}: 파일이 없습니다.")
        return
    blob = json.loads(path.read_text(encoding="utf-8"))

    art_cnt = par_cnt = qa_cnt = 0
    empty_ans = out_of_bounds = mismatch = 0
    bad_samples = []

    ctx_lengths = []
    ans_lengths = []

    for art in blob["data"]:
        art_cnt += 1
        for par in art["paragraphs"]:
            par_cnt += 1
            ctx = par["context"]
            ctx_lengths.append(len(ctx))
            for qa in par["qas"]:
                qa_cnt += 1
                if not qa["answers"]:
                    empty_ans += 1
                    bad_samples.append(("EMPTY", qa["id"], qa["question"][:40]))
                    continue
                ans = qa["answers"][0]
                text, start = ans["text"], ans["answer_start"]
                ans_lengths.append(len(text))

                if start < 0 or start + len(text) > len(ctx):
                    out_of_bounds += 1
                    bad_samples.append(("OOB", qa["id"], qa["question"][:40]))
                elif ctx[start:start + len(text)] != text:
                    mismatch += 1
                    bad_samples.append(("MISMATCH", qa["id"], qa["question"][:40]))

    print(f"\n📜 {path.name}")
    print(f"  articles        : {art_cnt}")
    print(f"  paragraphs      : {par_cnt}")
    print(f"  QAs             : {qa_cnt}")
    print(f"  ├── empty answer      : {empty_ans}")
    print(f"  ├── start OOB         : {out_of_bounds}")
    print(f"  └── substring mismatch: {mismatch}")

    if ctx_lengths:
        print(f"  context length  : max {max(ctx_lengths)}, "
              f"avg {statistics.mean(ctx_lengths):.1f}")
    if ans_lengths:
        print(f"  answer length   : max {max(ans_lengths)}, "
              f"avg {statistics.mean(ans_lengths):.1f}")

    if bad_samples:
        print("\n  ⚠️  첫 5개 오류 샘플")
        for kind, qid, qpreview in bad_samples[:5]:
            print(f"   [{kind}] {qid}  |  {qpreview}…")

# 실행
validate("train")
validate("val")


📜 train.json
  articles        : 1429
  paragraphs      : 1429
  QAs             : 2436
  ├── empty answer      : 0
  ├── start OOB         : 0
  └── substring mismatch: 0
  context length  : max 185806, avg 3079.5
  answer length   : max 63, avg 5.5

📜 val.json
  articles        : 276
  paragraphs      : 276
  QAs             : 490
  ├── empty answer      : 0
  ├── start OOB         : 0
  └── substring mismatch: 0
  context length  : max 129815, avg 2454.8
  answer length   : max 36, avg 5.5
